<a href="https://colab.research.google.com/github/nabilaAmir/python-codes-/blob/main/Bert_result.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 9.3MB/s 
     |████████████████████████████████| 3.2MB 33.6MB/s 
     |████████████████████████████████| 890kB 47.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=9c40c5e53ca915d4d8dde09e8e125f362cbf0d3b214f052b5419df77783cf3be
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import BertTokenizer, BertModel
from scipy.spatial.distance import cosine
import matplotlib.pyplot as plt
import json
import io
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# %matplotlib inline

# OPTIONAL: if you want to have more information on what's happening, activate the logger as follows
# import logging
# logging.basicConfig(level=logging.INFO)

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
# split into words
from nltk.tokenize import word_tokenize

In [ ]:
nltk.download('punkt')

from nltk import word_tokenize,sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from nltk.corpus import stopwords
stop_words=nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
stop_words = list(stopwords.words('english'))
#stopword = list(stopwords.words('english'))
#l = ['maggi','maggie','#maggi','#maggie']
#stopword = stopword + l
#print(stopword)
newStopWords = ['well','able','about','need','require','possible','poorly','possibly','potentially','present','presents','presented','primarily','primarily','provided','provides','put','rather','recently','regarding','regardless','related','research','This','respectively','results','resulting','second','secondly','several','show','shows','showed','shown','significantly','significant','since','slightly','somewhat','specifically','states','still','substantially','suggest','suggests','suggested','system','take','takes','taken','than','that','That','these','These','they','those','thousand','two','real','throughout','through','thus','till','toward','towards','trying','under','unfortunately','until','usefulness','value','where','while','within','words','would','Most',                'requires','using','In','for','needs','novel','obtained','only','otherwise','particular','poorly','provides','provide','found','information','mainly','makes','many','various','recent','year','years','merely','million','billion','most','mostly','much','namely','necessary','further','furthermore','getting','gets','give','gives','got','has','have','hence','however','ignored','ignore','importance','indicate','indicates','important','above','accordance','	according','accordingly','across','added','adopted','	affects','after','afterwards','against','all','	allow','almost','also','although','among','	amount','	amongst','and','another','apart','appear','apparently','appropriate','approximately','are','around','associated','available','because','before','beside','besides','best','beyond','between','briefly','brief','cannot','can','changes','come','concerning','consequently','therefore','considering','contain','contains','corresponding','correspond','could','currently','describe','despite','different','did','do','does','due','each','effect','affect','enough','entirely','entire','especially','find','followed','following','formerly','about','keep','keeps','according','accordance','knows', 'This','time','problem','found','The','step','obtain','three','We', 'paper','Paper','form','from','meet','meets','illustrate','beyond','known','it','It','last','recent','results','reveal','show','again','against','afterwards','also','moreover','like','makes','many','different','although','always','however','but','much','available','both','by','come','contains','therefore','do','does','during','especially','every','enough','particular','provide','provides','give','given','gives','hence','important','necessary','similar','similarly','still','sufficient','great','influence']
for i in newStopWords:
    stop_words.append(i)
import re

In [ ]:
#import codecs
import json
import io
data=json.load(io.open('Mind_dataset.json', 'r', encoding='utf-8-sig'))
#with codecs.open('Mind_dataset.json', 'r', encoding="utf-8") as f:
  #data = json.load(f)

In [ ]:
import tensorflow as tf
import numpy as np
y_list = []
ids_list= []
i=0
while i<len(data):
#while i<1:
  max_seq_length = 512
  paper_data=data[i]
  paper_Abstract=paper_data['Abstract']
  paper_Abstract = (paper_Abstract[:3000] + '..')
  #if len(paper_Abstract)>500 and language=='en':
   # paper_Abstract = (paper_Abstract[:3000] + '..')
  word_tokens = word_tokenize(paper_Abstract)

  filtered_sentence = [w for w in word_tokens if not w in stop_words]

  filtered_sentence = []

  for w in word_tokens:
    if w not in stop_words:
      filtered_sentence.append(w)

  output = str(filtered_sentence)
  output=output.strip("[ ]")
  output=output.strip("' '")
  re.sub(r'(?!(([^"]*"){2})*[^"]*$),', "", output)
  re.sub(r'"[^"]*"', lambda m: m.group(0).replace(',',''), output)
  output = output.replace("'","")
  output1 = output.replace(",","")
  abstract_text=re.sub(r" ?\([^)]+\)", "", output1)
# Add the special tokens.
  marked_text = "[CLS] " + abstract_text +" [SEP]"
# Split the sentence into tokens.
  tokenized_text = tokenizer.tokenize(marked_text)
# Map the token strings to their vocabulary indeces.
  indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
  if len(indexed_tokens) > 500:
    indexed_tokens = indexed_tokens[:(max_seq_length - 10)]
    #indexed_tokens = (indexed_tokens[:510] + '..')
# Mark each of the 22 tokens as belonging to sentence "1".
  segments_ids = [1] * len(tokenized_text)
# Convert inputs to PyTorch tensors
  tokens_tensor = torch.tensor([indexed_tokens])
  segments_tensors = torch.tensor([segments_ids])
# Put the model in "evaluation" mode, meaning feed-forward operation.
  model.eval()

# Run the text through BERT, and collect all of the hidden states produced
  with torch.no_grad():
      outputs = model(tokens_tensor, segments_tensors)
      hidden_states = outputs[2]

  token_embeddings = torch.stack(hidden_states, dim=0)
# Remove dimension 1, the "batches".
  token_embeddings = torch.squeeze(token_embeddings, dim=1)
# Swap dimensions 0 and 1.
  token_embeddings = token_embeddings.permute(1,0,2)
# Stores the token vectors, with shape [22 x 768]
  token_vecs_sum = []
# For each token in the sentence...
  for token in token_embeddings:
    # `token` is a [12 x 768] tensor

    # Sum the vectors from the last four layers.
      sum_vec = torch.sum(token[-4:], dim=0)

    # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)
# `token_vecs` is a tensor with shape [22 x 768]
  token_vecs = hidden_states[-2][0]
#token_vecs2 = hidden_states[-6][0]

# Calculate the average of all 22 token vectors.
  sentence_embedding = torch.mean(token_vecs, dim=0)
  # print(sentence_embedding)
  y_list.append(sentence_embedding)
  ids_list.append(paper_data['id'])
  i=i+1



In [ ]:
print(ids_list[0])
print(y_list[0])

N88753
tensor([ 5.5207e-01, -2.0130e-01,  9.1407e-01, -2.5093e-01,  2.9851e-01,
        -4.9025e-01, -2.0465e-02, -4.2026e-01, -1.2633e-01, -4.0398e-02,
        -1.2085e-01, -1.1418e-01, -2.4232e-01,  1.0129e-01, -3.4339e-01,
         5.5528e-01, -1.4585e-01,  4.0517e-01, -5.9505e-01,  3.2793e-01,
         4.9243e-01,  3.5403e-01, -1.4863e-01,  1.1211e-01,  5.5581e-01,
         9.5116e-03,  6.3436e-02, -5.3035e-01,  1.4585e-01,  1.6811e-01,
         9.1815e-01,  4.2981e-01,  3.6551e-01, -3.6338e-01,  3.2042e-01,
        -4.9964e-01, -4.8709e-01, -1.6620e-01,  4.0670e-02,  2.6005e-01,
        -5.9320e-01, -3.9129e-01,  2.7027e-01, -1.9529e-01,  2.4791e-02,
        -8.5172e-02, -1.1368e-01, -2.5673e-01,  2.9029e-01, -1.1164e-01,
        -5.8472e-01, -2.3970e-01, -3.6285e-01, -4.0847e-01,  6.8930e-01,
         8.9999e-02,  6.1885e-02, -5.1735e-01, -3.5798e-01, -4.2988e-02,
        -3.2106e-01, -2.5574e-01, -5.6341e-02,  9.6466e-02,  3.4540e-02,
        -1.0873e-01,  3.2835e-01,  8.3354e-0

In [ ]:
i=0
j=1
while i<len(data)-1:
  #print(paper1_id, paper2_id)
  for x in y_list:
    j=i+1
    while j <len(y_list):
      sentence_similarity = 1 - cosine(y_list[i], y_list[j])
      paper1_data=data[i]
      paper2_data=data[j]
      paper1_id=paper1_data['id']
      paper2_id=paper2_data['id']
      with open('embedding.txt', 'a+') as f:
        f.write("%s %s %s"  % (paper1_id, paper2_id, sentence_similarity))
        f.write("\n")
      #print(paper1_id,paper2_id,sentence_similarity)
      #print('another similarity')

      j=j+1
    i=i+1
  i=i+1

In [ ]:
sentence_similarity = 1-cosine(y_list[1], y_list[0])
print(round(sentence_similarity, 8))


0.81118399


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from dbn.tensorflow import SupervisedDBNClassification

In [ ]:
bert_embeddings = np.load('bert_embeddings.npy')
labels = np.load('labels.npy')

In [ ]:
scaler = StandardScaler()
bert_embeddings_scaled = scaler.fit_transform(bert_embeddings)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    bert_embeddings_scaled, labels, test_size=0.2, random_state=42
)

In [ ]:
dbn = SupervisedDBNClassification(
    hidden_layers_structure=[256, 128],  # Hidden layers with 256 and 128 neurons
    learning_rate_rbm=0.01,              # Learning rate for RBM pretraining
    learning_rate=0.01,                  # Learning rate for fine-tuning
    n_epochs_rbm=10,                     # Epochs for RBM training
    n_iter_backprop=100,                 # Iterations for backpropagation fine-tuning
    batch_size=32,                       # Batch size
    activation_function='relu',          # Activation function
    dropout_p=0.2                        # Dropout probability
)

In [ ]:
print("Training DBN...")
dbn.fit(X_train, y_train)

In [ ]:
print("Evaluating DBN...")
y_pred = dbn.predict(X_test)

In [ ]:
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {accuracy:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Step 6: Save the Trained Model
dbn.save('dbn_model.pkl')
print("Model saved to 'dbn_model.pkl'")